In [16]:
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import evaluate
from Bio import motifs
from Bio.Seq import Seq
import random
import requests

from ..src.motifs import fetch_all_motifs, save_to_csv

ImportError: attempted relative import with no known parent package

In [13]:

    # Example: Retrieve all high-quality motifs for vertebrates
    tax_group = "vertebrates"

    try:
        print(f"Fetching high-quality motifs for {tax_group}...")
        all_motifs = fetch_all_motifs(tax_group=tax_group)
        print(f"Retrieved {len(all_motifs)} motifs.")

        # Save motifs to a CSV file
        output_file = "high_quality_motifs_with_pfm_pwm.csv"
        save_to_csv(all_motifs, output_file)
        print(f"Saved motifs to {output_file}")
    except ValueError as e:
        print(e)

if __name__ == "__main__":
    main()



Fetching high-quality motifs for vertebrates...
Fetching motifs from: http://jaspar.genereg.net/api/v1/matrix/
Fetching motifs from: https://jaspar.elixir.no/api/v1/matrix/?collection=CORE&page=2&quality=high&tax_group=vertebrates
Fetching motifs from: https://jaspar.elixir.no/api/v1/matrix/?collection=CORE&page=3&quality=high&tax_group=vertebrates
Fetching motifs from: https://jaspar.elixir.no/api/v1/matrix/?collection=CORE&page=4&quality=high&tax_group=vertebrates
Fetching motifs from: https://jaspar.elixir.no/api/v1/matrix/?collection=CORE&page=5&quality=high&tax_group=vertebrates
Fetching motifs from: https://jaspar.elixir.no/api/v1/matrix/?collection=CORE&page=6&quality=high&tax_group=vertebrates
Fetching motifs from: https://jaspar.elixir.no/api/v1/matrix/?collection=CORE&page=7&quality=high&tax_group=vertebrates
Fetching motifs from: https://jaspar.elixir.no/api/v1/matrix/?collection=CORE&page=8&quality=high&tax_group=vertebrates
Fetching motifs from: https://jaspar.elixir.no/ap

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [14]:
all_motifs

NameError: name 'all_motifs' is not defined

In [13]:
import requests
from Bio import motifs
from io import StringIO

def get_jaspar_pwms(tax_group="vertebrates", collection="CORE"):
    """
    Fetch PWMs for a specific taxonomic group and collection from JASPAR.
    Args:
        tax_group (str): Taxonomic group, e.g., "vertebrates".
        collection (str): JASPAR collection, e.g., "CORE".
    Returns:
        dict: Dictionary of PWMs with transcription factor names as keys.
    """
    base_url = f"https://jaspar.genereg.net/api/v1/matrix/?tax_group={tax_group}&collection={collection}"
    response = requests.get(base_url)

    if response.status_code != 200:
        raise ValueError(f"API call failed with status code {response.status_code}")

    pwm_data = response.json()
    if not pwm_data.get('results'):
        raise ValueError("No PWM data found. Check query parameters.")

    pwms = {}
    for motif in pwm_data['results']:
        tf_name = motif.get('name', 'Unknown')
        pwm_url = motif.get('url')  # URL to fetch full motif data
        pwm_response = requests.get(pwm_url)

        if pwm_response.status_code == 200:
            motif_details = pwm_response.json()
            pfm = motif_details.get('pfm')
            if pfm:
                # Convert PFM (Position Frequency Matrix) to PWM
                pfm_string = "\n".join([" ".join(map(str, row)) for row in pfm])
                pwm = motifs.read(StringIO(pfm_string), "pfm")
                pwms[tf_name] = pwm
            else:
                print(f"No PFM found for {tf_name} (ID: {motif['matrix_id']})")
        else:
            print(f"Failed to fetch PWM for {tf_name} (URL: {pwm_url})")

    return pwms

# Test the function
jaspar_pwms = get_jaspar_pwms(tax_group="vertebrates", collection="CORE")
print("PWMs fetched:", list(jaspar_pwms.keys()))



PWMs fetched: ['ALX3', 'AR', 'ARGFX', 'ARNT2', 'ARNT::HIF1A', 'ASCL1']


In [9]:
base_url = "https://jaspar.genereg.net/api/v1/matrix/?tax_group=vertebrates&collection=CORE"
response = requests.get(base_url)
print(response.status_code)  # Check HTTP status
print(response.json())  

200
{'count': 1912, 'next': 'https://jaspar.elixir.no/api/v1/matrix/?collection=CORE&page=2&tax_group=vertebrates', 'previous': None, 'results': [{'matrix_id': 'MA0634.1', 'name': 'ALX3', 'collection': 'CORE', 'base_id': 'MA0634', 'version': '1', 'sequence_logo': 'https://jaspar.elixir.no/static/logos/svg/MA0634.1.svg', 'url': 'https://jaspar.elixir.no/api/v1/matrix/MA0634.1/'}, {'matrix_id': 'MA0634.2', 'name': 'ALX3', 'collection': 'CORE', 'base_id': 'MA0634', 'version': '2', 'sequence_logo': 'https://jaspar.elixir.no/static/logos/svg/MA0634.2.svg', 'url': 'https://jaspar.elixir.no/api/v1/matrix/MA0634.2/'}, {'matrix_id': 'MA0007.2', 'name': 'AR', 'collection': 'CORE', 'base_id': 'MA0007', 'version': '2', 'sequence_logo': 'https://jaspar.elixir.no/static/logos/svg/MA0007.2.svg', 'url': 'https://jaspar.elixir.no/api/v1/matrix/MA0007.2/'}, {'matrix_id': 'MA1463.1', 'name': 'ARGFX', 'collection': 'CORE', 'base_id': 'MA1463', 'version': '1', 'sequence_logo': 'https://jaspar.elixir.no/sta

In [10]:
data = response.json()
print(data.keys())           # Inspect top-level keys
print(data.get('results'))   # Check for PWM data

dict_keys(['count', 'next', 'previous', 'results'])
[{'matrix_id': 'MA0634.1', 'name': 'ALX3', 'collection': 'CORE', 'base_id': 'MA0634', 'version': '1', 'sequence_logo': 'https://jaspar.elixir.no/static/logos/svg/MA0634.1.svg', 'url': 'https://jaspar.elixir.no/api/v1/matrix/MA0634.1/'}, {'matrix_id': 'MA0634.2', 'name': 'ALX3', 'collection': 'CORE', 'base_id': 'MA0634', 'version': '2', 'sequence_logo': 'https://jaspar.elixir.no/static/logos/svg/MA0634.2.svg', 'url': 'https://jaspar.elixir.no/api/v1/matrix/MA0634.2/'}, {'matrix_id': 'MA0007.2', 'name': 'AR', 'collection': 'CORE', 'base_id': 'MA0007', 'version': '2', 'sequence_logo': 'https://jaspar.elixir.no/static/logos/svg/MA0007.2.svg', 'url': 'https://jaspar.elixir.no/api/v1/matrix/MA0007.2/'}, {'matrix_id': 'MA1463.1', 'name': 'ARGFX', 'collection': 'CORE', 'base_id': 'MA1463', 'version': '1', 'sequence_logo': 'https://jaspar.elixir.no/static/logos/svg/MA1463.1.svg', 'url': 'https://jaspar.elixir.no/api/v1/matrix/MA1463.1/'}, {'ma

In [ ]:
import random

def generate_sequences_from_pwm(pwm, num_sequences=1000, seq_length=30):
    """
    Generate synthetic sequences using a PWM.
    Args:
        pwm (Bio.motifs.Motif): PWM from JASPAR.
        num_sequences (int): Number of positive sequences to generate.
        seq_length (int): Length of generated sequences.
    Returns:
        tuple: Positive and negative sequences with labels.
    """
    sequences = []
    labels = []

    # Convert PWM into cumulative probabilities for efficient sampling
    cumulative_pwm = []
    for position in pwm.pwm:
        cumulative = [0]
        for base in "ACGT":
            cumulative.append(cumulative[-1] + pwm.pwm[position][base])
        cumulative_pwm.append(cumulative[1:])

    for _ in range(num_sequences):
        # Generate a positive sequence based on the PWM
        pos_seq = ""
        for probs in cumulative_pwm:
            rand_val = random.random()
            if rand_val < probs[0]:
                pos_seq += "A"
            elif rand_val < probs[1]:
                pos_seq += "C"
            elif rand_val < probs[2]:
                pos_seq += "G"
            else:
                pos_seq += "T"

        sequences.append(pos_seq)
        labels.append(1)  # Positive label

        # Generate a negative sequence by introducing random mutations
        neg_seq = ''.join(
            random.choice("ACGT") if random.random() < 0.1 else base
            for base in pos_seq
        )
        sequences.append(neg_seq)
        labels.append(0)  # Negative label

    return sequences, labels

# Example usage for one transcription factor
tf_name = "ALX3"  # Example TF
if tf_name in jaspar_pwms:
    pwm = jaspar_pwms[tf_name]
    sequences, labels = generate_sequences_from_pwm(pwm)
    print(f"Generated {len(sequences)} sequences for {tf_name}.")
else:
    print(f"Transcription factor {tf_name} not found in jaspar_pwms.")

TypeError: tuple indices must be integers or slices, not str

In [8]:
print(jaspar_pwms.keys())


dict_keys([])


In [9]:
# Load pretrained ESM model (e.g., ESM-2 from Hugging Face)
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")  # Replace with your ESM model

def preprocess_function(examples):
    return tokenizer(examples["sequence"], padding="max_length", truncation=True)

# Create Dataset
sequences, labels = generate_sequences_from_pwm(pwm=[0.3, 0.2, 0.3, 0.2])  # Replace with real PWM
data = {"sequence": sequences, "label": labels}
dataset = Dataset.from_dict(data)

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
model = AutoModelForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels=2)

# Adjust final layers for binary classification
model.config.problem_type = "single_label_classification"

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

# Define metrics
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset.train_test_split(test_size=0.1)["train"],
    eval_dataset=tokenized_dataset.train_test_split(test_size=0.1)["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

/home/anton.thieme/miniconda3/envs/sc_env/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/anton.thieme/miniconda3/envs/sc_env/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


/tmp/ipykernel_145328/4229830567.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.694100,0.692913,0.500000


KeyboardInterrupt: 